<div style="border: 2px solid #228be6; padding: 15px; border-radius: 10px; background-color: #f1f5f9; text-align: center; color: #1e293b; font-weight: bold;">

# 🚀 RETAIL INVENTORY MANAGEMENT SYSTEM - SCRUM IMPLEMENTATION USING LANGCHAIN

</div>

---

## **Setup and Imports**

To get started, we install the required packages:

```bash
!pip install -q langchain openai python-dotenv


In [24]:
!pip install -q langchain openai python-dotenv

import os
import re
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate




## 🔐 OpenAI API Key Setup

Set the OpenAI API key for `ChatOpenAI` model.


In [ ]:

# Set API key manually (OPTION 2)
openai_api_key = ""
llm = ChatOpenAI(model="gpt-4o", temperature=0.3, openai_api_key=openai_api_key)


## 🧠 Agent Class

Defines a reusable `Agent` class that encapsulates LangChain logic with role-based prompting.


In [26]:

# ========================================================================================
#  AGENT CLASS DEFINITION
# ========================================================================================
class Agent:
    def __init__(self, name, system_prompt, llm):
        self.name = name
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(system_prompt),
            HumanMessagePromptTemplate.from_template("{conversation_history}\n{input}")
        ])
        self.chain = LLMChain(llm=llm, prompt=self.prompt, verbose=False)

    def respond(self, input_text, history):
        return self.chain.predict(input=input_text, conversation_history=history)

MAX_HISTORY_TOKENS = 8000

def manage_history(history):
    if len(history) > MAX_HISTORY_TOKENS:
        return history[-MAX_HISTORY_TOKENS:]
    return history


## 🧑‍💼 Agent Role Prompts

System prompts for each Scrum role, including:
- Product Owner
- Business Analyst
- Developer
- QA Engineer
- etc.


In [27]:


# ========================================================================================
# 🧑‍💼 AGENT ROLE PROMPTS
# ========================================================================================
role_prompts = {
    "Customer_Proxy": '''You are a Customer Proxy agent acting as a mediator between the retail stakeholder and the AI Scrum team.

Your role is to clearly relay inventory management expectations, goals, and user pain points to the Product Owner.

Your retail business needs an **AI-driven inventory management system** that:
1. Tracks inventory levels in real-time.
2. Predicts stock shortages and suggests reorder points.
3. Automates reordering processes based on demand forecasting.
4. Supports multiple warehouse locations.
5. Generates comprehensive reports on stock movement.

At the end of every sprint, you will review and provide feedback for potential changes or approvals.''',

    "Product_Owner": '''You are a Product Owner in an Agile Scrum team.

### Responsibilities:
- Translate the **Customer Proxy's** high-level business goals into user stories and backlog items.
- Prioritize and maintain the product backlog.
- Collaborate with all Scrum roles to define sprint goals.
- Communicate customer feedback to the team.
- Ensure all sprint deliverables meet customer expectations.

### Deliverables:
- Product backlog with user stories.
- Sprint plan (sprint goals, estimates, priority).
- Feedback response and change plan.''',

    "Scrum_Master": '''You are the Scrum Master, a facilitator of Agile ceremonies and responsible for removing blockers.

### Responsibilities:
- Facilitate daily standups, sprint planning, sprint reviews, and retrospectives.
- Ensure Scrum principles are followed.
- Help the team improve velocity and remove impediments.
- Promote collaboration and communication among all agents.

### Deliverables:
- Summary of Scrum ceremonies.
- Identified blockers and mitigation plans.''',

    "Business_Analyst": '''You are a Business Analyst collaborating with the Product Owner to refine user stories.

### Responsibilities:
- Break down product backlog items into detailed requirements.
- Include acceptance criteria, user personas, and edge cases.
- Clarify ambiguous user needs.

### Deliverables:
- Detailed functional and non-functional requirements.
- Workload estimates and productivity planning.''',

    "Architect": '''You are a System Architect designing scalable solutions.

### Responsibilities:
- Convert user stories into architecture components.
- Produce diagrams (components, APIs, DB schemas).
- Identify risks and technical constraints.

### Deliverables:
- Architectural blueprint.
- Design metrics and rework plan if needed.''',

    "Developer": '''You are a Developer responsible for delivering features as per the sprint backlog.

### Responsibilities:
- Write modular, testable code.
- Estimate lines of code (SLOC), productivity, and sprint duration.
- Collaborate with QA, UI/UX, and PO.

### Deliverables:
- Feature implementation.
- Sprint burndown progress report.''',

    "UI_UX_Designer": '''You are a UI/UX Designer focused on user-centric interface design.

### Responsibilities:
- Create wireframes and high-fidelity designs for prioritized user stories.
- Incorporate accessibility and responsiveness.
- Estimate design effort and timeline.

### Deliverables:
- Screens per sprint.
- Design feedback incorporated from reviews.''',

    "QA_Engineer": '''You are a QA Engineer responsible for validating sprint deliverables.

### Responsibilities:
- Develop test cases based on user stories.
- Execute functional, regression, and integration tests.
- Report bugs with severity levels.

### Deliverables:
- Test coverage metrics.
- QA sign-off or defect backlog.''',

    "DevOps_Engineer": '''You are a DevOps Engineer handling CI/CD pipelines and environment readiness.

### Responsibilities:
- Build CI/CD pipelines for testing and deployment.
- Define infrastructure as code (IaC).
- Monitor build and deployment success.

### Deliverables:
- Deployment plan per sprint.
- Productivity and environment uptime metrics.''',

    "Security_Engineer": '''You are a Security Engineer safeguarding the system from vulnerabilities.

### Responsibilities:
- Conduct sprint-based security audits.
- Perform threat modeling and code scanning.
- Recommend risk mitigation.

### Deliverables:
- Security report per sprint.
- Checklist of patches or concerns.''',

    "Technical_Writer": '''You are a Technical Writer responsible for documenting sprint deliverables.

### Responsibilities:
- Write user guides, API docs, release notes.
- Collaborate with developers and testers for accuracy.
- Ensure documentation is clear and version-controlled.
- Provide task-wise estimates similar to technical project planning documents.
- Break down documentation tasks into: Writing, Review, Rework.
- Estimate hours using metrics like pages/hour and expected defects.

### Deliverables:
- A breakdown table for each document type (e.g., API doc, User Guide).
- Estimate: Pages, Hours, Review prep time, Review time, Defects/hour.
- Rework time based on defect rate.
- Total documentation time for the sprint.''',

    "Project_Summarizer": '''You are a Project Summarizer reviewing the end-to-end Scrum-based development of a Retail Inventory Management System.

Your job is to generate a concise and structured summary of the project based on the entire conversation history. Highlight the following:

-  Project goals
-  Key features discussed
-  Major design or technical decisions
-  QA outcomes
-  Customer feedback or requested changes
-  Final output and sprint status

Format your response clearly using bullet points or headings. Keep it brief but complete.'''
}

agents = {role: Agent(role, prompt, llm) for role, prompt in role_prompts.items()}
agents["Project_Summarizer"] = Agent("Project_Summarizer", role_prompts["Project_Summarizer"], llm)

<div style="border: 2px solid #0f766e; padding: 15px; border-radius: 10px; background-color: #ecfdf5; text-align: center; color: #064e3b; font-weight: bold;">

# SPRINT EXECUTION (WITH MANUAL CUSTOMER INTERACTION)

</div>


##  **Sprint Simulation Workflow**

This section walks through a **complete Agile sprint cycle**, where each AI agent performs role-specific actions based on current sprint context. The customer provides real-time feedback at the end of the cycle, allowing adaptive rework if necessary.

###  **Initial Customer Input**


In [28]:

conversation_history = ""
print("================================================================================")
print("🧑‍💼 CUSTOMER REQUIREMENTS PHASE")
print("================================================================================")
initial_input = input(" Please enter the type of inventory system you want to build: ")
conversation_history = f"Customer_Proxy: {initial_input}\n"

sprint_order = [
    "Product_Owner", "Business_Analyst", "Architect", "Developer", "UI_UX_Designer",
    "QA_Engineer", "DevOps_Engineer", "Security_Engineer", "Technical_Writer",
    "Scrum_Master", "Customer_Proxy"
]


🧑‍💼 CUSTOMER REQUIREMENTS PHASE


## 🔁 Role-by-Role Execution

Loop through each role in the Scrum workflow and generate responses.


In [ ]:

for role in sprint_order:
    print("================================================================================")
    print(f"🧑‍💻 {role.upper()} PHASE")
    print("================================================================================")
    task_prompt = role_prompts[role]
    try:
        if role == "Customer_Proxy":
            print("\n Awaiting feedback from the customer...")
            response = input("🗣️  Enter feedback from Customer Proxy (e.g., 'Looks good', 'Please rework feature X'): ")
        else:
            response = agents[role].respond(task_prompt, conversation_history)

        print("--------------------------------------------------------------------------------")
        print(f"🤖 {role} RESPONSE:\n{response}")
        print("================================================================================")
        conversation_history += f"{role}: {response}\n"
        conversation_history = manage_history(conversation_history)

        if role == "Customer_Proxy":
            if any(x in response.lower() for x in ["approved", "looks good", "happy"]):
                print(" Customer approved. Sprint is complete.")
                break
            elif any(x in response.lower() for x in ["change", "adjust", "rework"]):
                print(" Customer requested rework. Restarting QA phase.")
                sprint_order.insert(sprint_order.index("QA_Engineer"), "QA_Engineer")
    except Exception as e:
        print(f" Error during {role} execution: {e}")
        break

# ========================================================================================
# 📘 END OF SPRINT
# ========================================================================================
print("\n SPRINT WORKFLOW COMPLETED.")

print("\n================================================================================")
print("📊 PROJECT SUMMARY PHASE")
print("================================================================================")
try:
    summary = agents["Project_Summarizer"].respond("Please summarize the entire project.", conversation_history)
    print("--------------------------------------------------------------------------------")
    print("📘 FINAL PROJECT SUMMARY:\n")
    print(summary)
    print("================================================================================")

except Exception as e:
    print(f" Error generating project summary: {e}")

🧑‍💻 PRODUCT_OWNER PHASE
--------------------------------------------------------------------------------
🤖 Product_Owner RESPONSE:
### Product Backlog with User Stories:

1. **User Story 1: Ergonomic Design**
   - As a customer, I want chairs with ergonomic designs to ensure comfort during long periods of sitting.
   - Acceptance Criteria:
     - Adjustable seat height and armrests.
     - Lumbar support for lower back.
     - Breathable material for seat and backrest.

2. **User Story 2: Durable Materials**
   - As a customer, I want chairs made from durable materials to ensure longevity and withstand daily use.
   - Acceptance Criteria:
     - High-quality frame material (e.g., steel or reinforced plastic).
     - Fabric that is resistant to wear and tear.
     - Warranty of at least 5 years.

3. **User Story 3: Aesthetic Appeal**
   - As a customer, I want chairs that are aesthetically pleasing to enhance the look of my space.
   - Acceptance Criteria:
     - Available in multiple c